# Load Data từ Silver Layer sang Gold Layer (Dimension & Fact Tables)

Notebook này sẽ load dữ liệu từ Silver layer và transform sang Gold layer với:
- Dimension tables có surrogate key tự động tăng
- Fact tables với các foreign keys tương ứng

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime

# Khởi tạo Spark Session
spark = (
    SparkSession.builder.appName("Load_Silver_To_Gold")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

# Tắt log WARN - chỉ hiển thị ERROR
spark.sparkContext.setLogLevel("ERROR")

print("Spark Session đã được khởi tạo!")
print("Log level đã được set thành ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 05:37:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/06 05:37:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/06 05:37:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark Session đã được khởi tạo!
Log level đã được set thành ERROR


## 2. Load Dimension Tables từ Silver Layer

### 2.1. Dim_Time - Tạo bảng thời gian từ dữ liệu benchmark

In [2]:
print("Đang load Dim_Time...")

# Đọc dữ liệu từ Silver
df_benchmark = spark.table("nessie.silver_tables.benchmark")

# Tạo dim_time từ các year unique trong benchmark
df_time = df_benchmark.select("year").distinct()

# Tạo day, month từ year (giả sử ngày 1/1 của mỗi năm)
df_time = df_time.withColumn("day", lit(1)) \
    .withColumn("month", lit(1))

# Tạo timeKey tự động tăng
window_spec = Window.orderBy("year")
df_time = df_time.withColumn("timeKey", row_number().over(window_spec))

# Sắp xếp lại cột theo thứ tự
df_dim_time = df_time.select("timeKey", "day", "month", "year")

# Ghi vào Gold layer
df_dim_time.writeTo("nessie.gold_tables.dim_time") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_time.count()} dòng vào dim_time")
df_dim_time.show(10)

Đang load Dim_Time...


Đã load 5 dòng vào dim_time
+-------+---+-----+----+
|timeKey|day|month|year|
+-------+---+-----+----+
|      1|  1|    1|2021|
|      2|  1|    1|2022|
|      3|  1|    1|2023|
|      4|  1|    1|2024|
|      5|  1|    1|2025|
+-------+---+-----+----+



### 2.2. Dim_Region - Bảng khu vực

In [3]:
print("Đang load Dim_Region...")

# Đọc dữ liệu từ Silver
df_region_silver = spark.table("nessie.silver_tables.region")

# Tạo regionKey tự động tăng
window_spec = Window.orderBy("regionId")
df_dim_region = df_region_silver.withColumn("regionKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_region = df_dim_region.select("regionKey", "regionId", "regionName")

# Ghi vào Gold layer
df_dim_region.writeTo("nessie.gold_tables.dim_region") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_region.count()} dòng vào dim_region")
df_dim_region.show(10)

Đang load Dim_Region...
Đã load 64 dòng vào dim_region
+---------+--------+--------------------+
|regionKey|regionId|          regionName|
+---------+--------+--------------------+
|        1|      01|      Sở GDĐT Hà Nội|
|        2|      02|Sở GDĐT TP. Hồ Ch...|
|        3|      03|   Sở GDĐT Hải Phòng|
|        4|      04|     Sở GDĐT Đà Nẵng|
|        5|      05|    Sở GDĐT Hà Giang|
|        6|      06|    Sở GDĐT Cao Bằng|
|        7|      07|    Sở GDĐT Lai Châu|
|        8|      08|     Sở GDĐT Lào Cai|
|        9|      09| Sở GDĐT Tuyên Quang|
|       10|      10|    Sở GDĐT Lạng Sơn|
+---------+--------+--------------------+
only showing top 10 rows



### 2.3. Dim_School - Bảng trường đại học

In [4]:
print("Đang load Dim_School...")

# Đọc dữ liệu từ Silver
df_school_silver = spark.table("nessie.silver_tables.school")

# Tạo schoolKey tự động tăng
window_spec = Window.orderBy("schoolId")
df_dim_school = df_school_silver.withColumn("schoolKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_school = df_dim_school.select("schoolKey", "schoolId", "schoolName", "province")

# Ghi vào Gold layer
df_dim_school.writeTo("nessie.gold_tables.dim_school") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_school.count()} dòng vào dim_school")
df_dim_school.show(10)

Đang load Dim_School...
Đã load 270 dòng vào dim_school
+---------+--------+--------------------+---------+
|schoolKey|schoolId|          schoolName| province|
+---------+--------+--------------------+---------+
|        1|   \bDMT|Phân hiệu ĐH Tài ...|Thanh Hóa|
|        2|     ANH|Học viện An Ninh ...|   Hà Nội|
|        3|     ANS|Đại học An Ninh N...|   Hà Nội|
|        4|     BKA|Đại học Bách khoa...|   Hà Nội|
|        5|     BMU|Đại học Buôn Ma T...|  Đắk Lắk|
|        6|     BPH| Học viện Biên Phòng|   Hà Nội|
|        7|     BVH|Học viện Công ngh...|   Hà Nội|
|        8|     BVS|Học viện Công ngh...|   TP HCM|
|        9|     CCM|Đại học Công Nghi...|   Hà Nội|
|       10|     CEA|Đại học Kinh Tế N...|  Nghệ An|
+---------+--------+--------------------+---------+
only showing top 10 rows



### 2.4. Dim_Major - Bảng ngành học

In [5]:
print("Đang load Dim_Major...")

# Đọc dữ liệu từ Silver
df_major_silver = spark.table("nessie.silver_tables.major")

# Tạo majorKey tự động tăng
window_spec = Window.orderBy("majorId")
df_dim_major = df_major_silver.withColumn("majorKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_major = df_dim_major.select("majorKey", "majorId", "majorName")

# Ghi vào Gold layer
df_dim_major.writeTo("nessie.gold_tables.dim_major") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_major.count()} dòng vào dim_major")
df_dim_major.show(10)

Đang load Dim_Major...
Đã load 3265 dòng vào dim_major
+--------+-------+--------------------+
|majorKey|majorId|           majorName|
+--------+-------+--------------------+
|       1|    106|   Khoa học Máy tính|
|       2|    107|   Kỹ thuật Máy tính|
|       3|    108|Điện - Điện tử - ...|
|       4|    109|     Kỹ Thuật Cơ khí|
|       5|    110| Kỹ Thuật Cơ Điện tử|
|       6|    112|           Dệt - May|
|       7|    114|Hoá - Thực phẩm -...|
|       8|    115|Xây dựng và Quản ...|
|       9|    117|           Kiến Trúc|
|      10|    120|  Dầu khí - Địa chất|
+--------+-------+--------------------+
only showing top 10 rows



### 2.5. Dim_Subject - Bảng môn học

In [6]:
print("Đang load Dim_Subject...")

# Đọc dữ liệu từ Silver
df_subject_silver = spark.table("nessie.silver_tables.subject")

# Tạo subjectKey tự động tăng (sử dụng subjectId làm sort key)
window_spec = Window.orderBy("subjectId")
df_dim_subject = df_subject_silver.withColumn("subjectKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_subject = df_dim_subject.select("subjectKey", "subjectId", "subjectName")

# Ghi vào Gold layer
df_dim_subject.writeTo("nessie.gold_tables.dim_subject") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_subject.count()} dòng vào dim_subject")
df_dim_subject.show(10)

Đang load Dim_Subject...
Đã load 88 dòng vào dim_subject
+----------+---------+--------------------+
|subjectKey|subjectId|         subjectName|
+----------+---------+--------------------+
|         1|        1|Biểu diễn nghệ thuật|
|         2|        2|Công nghệ công ng...|
|         3|        3|Công nghệ nông ng...|
|         4|        4|             GDKT&PL|
|         5|        5|              GDKTPL|
|         6|        6|                 Hoá|
|         7|        7|                 Hát|
|         8|        8|Hát hoặc biểu diễ...|
|         9|        9|            Hình họa|
|        10|       10|                 Hóa|
+----------+---------+--------------------+
only showing top 10 rows



### 2.6. Dim_Subject_Group - Bảng khối thi

In [7]:
print("Đang load Dim_Subject_Group...")

# Đọc dữ liệu từ Silver
df_subject_group_silver = spark.table("nessie.silver_tables.subject_group")

# Tạo subjectGroupKey tự động tăng
window_spec = Window.orderBy("subjectGroupId")
df_dim_subject_group = df_subject_group_silver.withColumn("subjectGroupKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_subject_group = df_dim_subject_group.select(
    "subjectGroupKey", 
    "subjectGroupId", 
    "subjectGroupName", 
    "subjectCombination"
)

# Ghi vào Gold layer
df_dim_subject_group.writeTo("nessie.gold_tables.dim_subject_group") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_subject_group.count()} dòng vào dim_subject_group")
df_dim_subject_group.show(10)

Đang load Dim_Subject_Group...
Đã load 212 dòng vào dim_subject_group
+---------------+--------------+----------------+--------------------+
|subjectGroupKey|subjectGroupId|subjectGroupName|  subjectCombination|
+---------------+--------------+----------------+--------------------+
|              1|             1|             D01|Toán-Ngữ Văn-Tiến...|
|              2|             2|             A00| Toán-Vật lí-Hóa học|
|              3|             3|             A01|Toán-Vật lí-Tiếng...|
|              4|             4|             D07|Toán-Hóa học-Tiến...|
|              5|             5|             B00|Toán-Hóa học-Sinh...|
|              6|             6|             C00|Ngữ văn-Lịch sử-Đ...|
|              7|             7|             C01| Ngữ văn-Toán-Vật lí|
|              8|             8|             D14|Ngữ văn-Lịch sử-T...|
|              9|             9|             D15|Ngữ văn-Địa lí-Ti...|
|             10|            10|             A02|Toán-Vật lí-Sinh học|
+------

### 2.7. Dim_Selection_Method - Bảng phương thức xét tuyển

In [8]:
print("Đang load Dim_Selection_Method...")

# Đọc dữ liệu từ Silver
df_selection_method_silver = spark.table("nessie.silver_tables.selection_method")

# Tạo selectionMethodKey tự động tăng
window_spec = Window.orderBy("selectionMethodId")
df_dim_selection_method = df_selection_method_silver.withColumn("selectionMethodKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_selection_method = df_dim_selection_method.select(
    "selectionMethodKey", 
    "selectionMethodId", 
    "selectionMethodName"
)

# Ghi vào Gold layer
df_dim_selection_method.writeTo("nessie.gold_tables.dim_selection_method") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_selection_method.count()} dòng vào dim_selection_method")
df_dim_selection_method.show(10)

Đang load Dim_Selection_Method...
Đã load 13 dòng vào dim_selection_method
+------------------+-----------------+--------------------+
|selectionMethodKey|selectionMethodId| selectionMethodName|
+------------------+-----------------+--------------------+
|                 1|                1|Điểm chuẩn theo p...|
|                 2|                2|Điểm chuẩn theo p...|
|                 3|                3|Điểm chuẩn theo p...|
|                 4|                4|Điểm chuẩn theo p...|
|                 5|                5|Điểm chuẩn theo p...|
|                 6|                6|Điểm chuẩn theo p...|
|                 7|                7|Điểm chuẩn theo p...|
|                 8|                8|Điểm chuẩn theo p...|
|                 9|                9|Điểm chuẩn theo p...|
|                10|               10|Điểm chuẩn theo p...|
+------------------+-----------------+--------------------+
only showing top 10 rows



## 3. Load Fact Tables từ Silver Layer

### 3.1. Fact_Benchmark - Load và join với các Dimension tables

In [9]:
print("Đang load Fact_Benchmark...")

# Đọc dữ liệu từ Silver
df_benchmark_silver = spark.table("nessie.silver_tables.benchmark")

# Đọc các dimension tables từ Gold để join
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_school_gold = spark.table("nessie.gold_tables.dim_school")
df_dim_major_gold = spark.table("nessie.gold_tables.dim_major")
df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")
df_dim_selection_method_gold = spark.table("nessie.gold_tables.dim_selection_method")

# Join từng bước để tránh Cartesian product
# Bước 1: Join với dim_time
df_fact = df_benchmark_silver.join(
    df_dim_time_gold, 
    df_benchmark_silver["year"] == df_dim_time_gold["year"], 
    "left"
).drop(df_dim_time_gold["year"])

# Bước 2: Join với dim_school
df_fact = df_fact.join(
    df_dim_school_gold, 
    df_fact["schoolId"] == df_dim_school_gold["schoolId"], 
    "left"
).drop(df_dim_school_gold["schoolId"])

# Bước 3: Join với dim_major
df_fact = df_fact.join(
    df_dim_major_gold, 
    df_fact["majorId"] == df_dim_major_gold["majorId"], 
    "left"
).drop(df_dim_major_gold["majorId"])

# Bước 4: Join với dim_subject_group
df_fact = df_fact.join(
    df_dim_subject_group_gold, 
    df_fact["subjectGroupId"] == df_dim_subject_group_gold["subjectGroupId"], 
    "left"
).drop(df_dim_subject_group_gold["subjectGroupId"])

# Bước 5: Join với dim_selection_method
df_fact = df_fact.join(
    df_dim_selection_method_gold, 
    df_fact["selectionMethodId"] == df_dim_selection_method_gold["selectionMethodId"], 
    "left"
).drop(df_dim_selection_method_gold["selectionMethodId"])

print(f"Sau tất cả join: {df_fact.count():,} dòng")

# Tạo benchmarkKey tự động tăng
window_spec = Window.orderBy("timeKey", "schoolKey", "majorKey")
df_fact = df_fact.withColumn("benchmarkKey", row_number().over(window_spec))

# Tính toán yearlyScoreGap
window_year_spec = Window.partitionBy("schoolKey", "majorKey", "selectionMethodKey").orderBy("timeKey")
df_fact = df_fact.withColumn(
    "yearlyScoreGap",
    coalesce(col("score") - lag("score", 1).over(window_year_spec), lit(0))
)

# Tính điểm trung bình cho mỗi ngành theo từng phương thức
window_avg_spec = Window.partitionBy("timeKey", "schoolKey", "majorKey", "selectionMethodKey")
df_fact = df_fact.withColumn(
    "avgScoreByMajor",
    avg("score").over(window_avg_spec)
)

# Xếp hạng ngành trong trường
window_major_in_school_spec = Window.partitionBy("timeKey", "schoolKey", "selectionMethodKey").orderBy(desc("avgScoreByMajor"), "majorKey")
df_fact = df_fact.withColumn("rankAmongMajor", dense_rank().over(window_major_in_school_spec))

# Xếp hạng ngành so với các trường khác
window_major_across_schools_spec = Window.partitionBy("timeKey", "majorKey", "selectionMethodKey").orderBy(desc("avgScoreByMajor"), "schoolKey")
df_fact = df_fact.withColumn("rankAmongSchools", dense_rank().over(window_major_across_schools_spec))

# Select các cột cuối cùng
df_fact_benchmark = df_fact.select(
    "benchmarkKey",
    "subjectGroupKey",
    "timeKey",
    "majorKey",
    "schoolKey",
    "selectionMethodKey",
    col("score").cast("float"),
    col("avgScoreByMajor").cast("float"),
    col("yearlyScoreGap").cast("float"),
    "rankAmongMajor",
    "rankAmongSchools"
)

# Ghi vào Gold layer
df_fact_benchmark.writeTo("nessie.gold_tables.fact_benchmark") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_benchmark.count()} dòng vào fact_benchmark")
df_fact_benchmark.show(10)

Đang load Fact_Benchmark...
Sau tất cả join: 143,863 dòng


Đã load 143863 dòng vào fact_benchmark
+------------+---------------+-------+--------+---------+------------------+-----+---------------+--------------+--------------+----------------+
|benchmarkKey|subjectGroupKey|timeKey|majorKey|schoolKey|selectionMethodKey|score|avgScoreByMajor|yearlyScoreGap|rankAmongMajor|rankAmongSchools|
+------------+---------------+-------+--------+---------+------------------+-----+---------------+--------------+--------------+----------------+
|       14165|              1|      1|      41|      206|                 3| 26.7|           26.7|           0.0|            14|               1|
|       14166|              4|      1|      41|      206|                 3| 26.7|           26.7|           0.0|            14|               1|
|       14167|              2|      1|      41|      206|                 3| 26.7|           26.7|           0.0|            14|               1|
|       14168|              3|      1|      41|      206|                 3| 26.7|   

### 3.2. Fact_Score_Distribution_By_Subject - Phân bố điểm theo môn

In [10]:
print("Đang load Fact_Score_Distribution_By_Subject...")

# Đọc dữ liệu từ Silver
df_student_scores = spark.table("nessie.silver_tables.student_scores")

# Đọc dimension tables
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
df_dim_subject_gold = spark.table("nessie.gold_tables.dim_subject")

# Explode map scores thành từng dòng (subjectId - điểm)
# Vì scores giờ là Map<Int, Float> (subjectId -> score)
# subjectId ở đây là INTEGER, chính là cột subjectId từ bảng silver.subject
df_scores_exploded = df_student_scores.select(
    "regionId",
    "year",
    explode("scores").alias("subjectId", "score")  # subjectId = key của map (Int)
)

# Phân loại điểm vào các mốc điểm (0.2, 0.4, 0.6, ..., 9.8, 10.0)
# Làm tròn điểm đến bội số của 0.2 gần nhất
df_scores_exploded = df_scores_exploded.withColumn(
    "scoreThreshold",
    (floor(col("score") / 0.2) * 0.2).cast("float")
)

# Đếm số lượng học sinh theo từng môn, khu vực, năm và ngưỡng điểm
df_distribution = df_scores_exploded.groupBy(
    "regionId",
    "subjectId",  # Đây là subjectId (Int) từ scores map
    "year",
    "scoreThreshold"
).agg(
    count("*").alias("quantity")
)

# Join với các dimension tables
# Join với dim_subject bằng subjectId để lấy subjectKey
df_fact = df_distribution \
    .join(df_dim_region_gold, df_distribution["regionId"] == df_dim_region_gold["regionId"], "left") \
    .join(df_dim_subject_gold, df_distribution["subjectId"] == df_dim_subject_gold["subjectId"], "left") \
    .join(df_dim_time_gold, df_distribution["year"] == df_dim_time_gold["year"], "left")

# Tạo SDBSKey tự động tăng
window_spec = Window.orderBy("timeKey", "regionKey", "subjectKey", "scoreThreshold")
df_fact = df_fact.withColumn("SDBSKey", row_number().over(window_spec))

# Select các cột cuối cùng
# Lưu ý: Lấy subjectKey từ df_dim_subject_gold (không phải subjectId)
df_fact_score_dist_subject = df_fact.select(
    "SDBSKey",
    df_dim_region_gold["regionKey"],
    df_dim_subject_gold["subjectKey"],  # subjectKey từ dim_subject (surrogate key)
    df_dim_time_gold["timeKey"],
    col("scoreThreshold").cast("float"),
    col("quantity").cast("float")
)

# Ghi vào Gold layer
df_fact_score_dist_subject.writeTo("nessie.gold_tables.fact_score_distribution_by_subject") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_score_dist_subject.count()} dòng vào fact_score_distribution_by_subject")
df_fact_score_dist_subject.show(10)

Đang load Fact_Score_Distribution_By_Subject...


Đã load 13213 dòng vào fact_score_distribution_by_subject
+-------+---------+----------+-------+--------------+--------+
|SDBSKey|regionKey|subjectKey|timeKey|scoreThreshold|quantity|
+-------+---------+----------+-------+--------------+--------+
|      1|        1|        10|      1|           0.0|     2.0|
|      2|        1|        10|      1|           0.2|     1.0|
|      3|        1|        10|      1|           0.4|     1.0|
|      4|        1|        10|      1|           0.6|     5.0|
|      5|        1|        10|      1|           1.0|    11.0|
|      6|        1|        10|      1|           1.2|    30.0|
|      7|        1|        10|      1|           1.4|    36.0|
|      8|        1|        10|      1|           1.6|   102.0|
|      9|        1|        10|      1|           2.0|   158.0|
|     10|        1|        10|      1|           2.2|   236.0|
+-------+---------+----------+-------+--------------+--------+
only showing top 10 rows



### 3.3. Fact_Score_Distribution_By_Subject_Group - Phân bố điểm theo khối thi

In [11]:
# print("Đang load Fact_Score_Distribution_By_Subject_Group...")

# # Đọc dữ liệu từ Silver
# df_student_scores = spark.table("nessie.silver_tables.student_scores")
# df_subject_group = spark.table("nessie.silver_tables.subject_group")
# df_subject = spark.table("nessie.silver_tables.subject")

# # Đọc dimension tables
# df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
# df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
# df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")

# # Tạo mapping subjectName -> subjectId
# subject_name_to_id = {row['subjectName']: row['subjectId'] for row in df_subject.collect()}
# print(f"Subject mapping: {subject_name_to_id}")

# # Broadcast mappings
# from pyspark.sql.types import FloatType, MapType, StringType
# subject_map_broadcast = spark.sparkContext.broadcast(subject_name_to_id)

# # Collect subject groups thành dictionary: {subjectGroupId: [list of subject names]}
# subject_groups_dict = {}
# for row in df_subject_group.collect():
#     group_id = row['subjectGroupId']
#     subjects = [s.strip() for s in row['subjectCombination'].split('-')]
#     subject_groups_dict[group_id] = subjects

# subject_groups_broadcast = spark.sparkContext.broadcast(subject_groups_dict)

# print(f"Số lượng khối thi: {len(subject_groups_dict)}")

# # UDF để tạo mapping Map<subjectGroupId, totalScore> từ scores của thí sinh
# def create_subject_group_scores_map(scores_map):
#     """
#     Từ scores_map (Map<subjectId, score>) của thí sinh,
#     tạo ra Map<subjectGroupId, totalScore> cho tất cả khối thi mà thí sinh có đủ môn
#     """
#     if scores_map is None:
#         return {}
    
#     subject_name_to_id_map = subject_map_broadcast.value
#     subject_id_to_name_map = {v: k for k, v in subject_name_to_id_map.items()}
#     subject_groups = subject_groups_broadcast.value
    
#     result = {}
    
#     # Với mỗi khối thi, kiểm tra xem thí sinh có đủ môn không
#     for group_id, required_subjects in subject_groups.items():
#         total_score = 0.0
#         has_all_subjects = True
        
#         for subject_name in required_subjects:
#             subject_id = subject_name_to_id_map.get(subject_name)
            
#             if subject_id is None or subject_id not in scores_map:
#                 has_all_subjects = False
#                 break
            
#             total_score += scores_map[subject_id]
        
#         # Chỉ thêm vào result nếu thí sinh có đủ tất cả các môn trong khối
#         if has_all_subjects:
#             result[group_id] = total_score
    
#     return result

# create_group_scores_udf = udf(create_subject_group_scores_map, MapType(StringType(), FloatType()))

# # Tạo cột mới chứa mapping subjectGroupId -> totalScore
# print("Đang tạo cột subject_group_scores cho student_scores...")
# df_student_with_group_scores = df_student_scores.withColumn(
#     "subject_group_scores",
#     create_group_scores_udf(col("scores"))
# )

# # Hiển thị sample
# print("Sample student_scores với subject_group_scores:")
# df_student_with_group_scores.select("studentId", "regionId", "year", "subject_group_scores").show(5, truncate=False)

# # Explode map subject_group_scores thành từng dòng (subjectGroupId - totalScore)
# df_scores_exploded = df_student_with_group_scores.select(
#     "regionId",
#     "year",
#     explode("subject_group_scores").alias("subjectGroupId", "totalScore")
# )

# print(f"Tổng số dòng sau explode: {df_scores_exploded.count():,}")

# # Phân loại điểm vào các khoảng (0-1, 1-2, ..., 29-30)
# df_scores_exploded = df_scores_exploded.withColumn(
#     "scoreRange",
#     floor(col("totalScore")).cast("float")
# )

# # Đếm số lượng học sinh theo từng khối thi, khu vực, năm và khoảng điểm
# df_distribution = df_scores_exploded.groupBy(
#     "regionId",
#     "subjectGroupId",
#     "year",
#     "scoreRange"
# ).agg(
#     count("*").alias("quantity")
# )

# print(f"Số dòng distribution: {df_distribution.count():,}")

# # Tạo mapping dictionaries từ dimension tables để tra cứu nhanh (thay vì JOIN)
# print("Đang tạo mapping dictionaries...")
# region_id_to_key = {row['regionId']: row['regionKey'] for row in df_dim_region_gold.select("regionId", "regionKey").collect()}
# subject_group_id_to_key = {row['subjectGroupId']: row['subjectGroupKey'] for row in df_dim_subject_group_gold.select("subjectGroupId", "subjectGroupKey").collect()}
# year_to_time_key = {row['year']: row['timeKey'] for row in df_dim_time_gold.select("year", "timeKey").collect()}

# # Broadcast mappings
# region_map_bc = spark.sparkContext.broadcast(region_id_to_key)
# subject_group_map_bc = spark.sparkContext.broadcast(subject_group_id_to_key)
# year_map_bc = spark.sparkContext.broadcast(year_to_time_key)

# # UDF để map các natural keys sang surrogate keys
# def map_to_keys(regionId, subjectGroupId, year):
#     region_key = region_map_bc.value.get(regionId)
#     subject_group_key = subject_group_map_bc.value.get(subjectGroupId)
#     time_key = year_map_bc.value.get(year)
#     return (region_key, subject_group_key, time_key)

# map_keys_udf = udf(map_to_keys, StructType([
#     StructField("regionKey", IntegerType(), True),
#     StructField("subjectGroupKey", IntegerType(), True),
#     StructField("timeKey", IntegerType(), True)
# ]))

# # Áp dụng mapping
# df_with_keys = df_distribution.withColumn(
#     "keys",
#     map_keys_udf(col("regionId"), col("subjectGroupId"), col("year"))
# )

# # Extract các keys từ struct
# df_with_keys = df_with_keys.withColumn("regionKey", col("keys.regionKey")) \
#     .withColumn("subjectGroupKey", col("keys.subjectGroupKey")) \
#     .withColumn("timeKey", col("keys.timeKey")) \
#     .drop("keys", "regionId", "subjectGroupId", "year")

# # Tạo SDBSGKey tự động tăng
# window_spec = Window.orderBy("timeKey", "regionKey", "subjectGroupKey", "scoreRange")
# df_with_keys = df_with_keys.withColumn("SDBSGKey", row_number().over(window_spec))

# # Select các cột cuối cùng
# df_fact_score_dist_group = df_with_keys.select(
#     "SDBSGKey",
#     "regionKey",
#     "subjectGroupKey",
#     "timeKey",
#     col("scoreRange").cast("float"),
#     col("quantity").cast("float")
# )

# # Ghi vào Gold layer
# df_fact_score_dist_group.writeTo("nessie.gold_tables.fact_score_distribution_by_subject_group") \
#     .using("iceberg") \
#     .partitionedBy("timeKey") \
#     .createOrReplace()

# print(f"Đã load {df_fact_score_dist_group.count()} dòng vào fact_score_distribution_by_subject_group")
# df_fact_score_dist_group.show(10)

#### 📝 PHẦN 1: Tính toán tất cả tổ hợp khối thi mà mỗi thí sinh có thể xét

**Mục tiêu:** Với mỗi thí sinh, tính điểm tổng cho TẤT CẢ các khối thi mà thí sinh có đủ điểm các môn.

In [ ]:
print("=" * 80)
print("PHẦN 1: TÍNH TOÁN TỔ HỢP KHỐI THI CHO MỖI THÍ SINH (CÓ DEBUG LOG)")
print("=" * 80)

# ==================== ĐỌC DỮ LIỆU ====================
print("\n[Bước 1] Đọc dữ liệu từ Silver layer...")
df_student_scores = spark.table("nessie.silver_tables.student_scores")
df_subject_group = spark.table("nessie.silver_tables.subject_group")
df_subject = spark.table("nessie.silver_tables.subject")

print(f"  ✓ Số lượng thí sinh: {df_student_scores.count():,}")
print(f"  ✓ Số lượng khối thi: {df_subject_group.count()}")
print(f"  ✓ Số lượng môn học: {df_subject.count()}")

# ==================== TẠO MAPPING DICTIONARIES ====================
print("\n[Bước 2] Tạo mapping dictionaries...")

# Mapping: Tên môn → ID môn (VD: "Toán" → 1)
subject_name_to_id = {row['subjectName']: row['subjectId'] for row in df_subject.collect()}
print(f"  ✓ Subject mapping: {subject_name_to_id}")

# Mapping: ID khối → Danh sách môn (VD: "A00" → ["Toán", "Lý", "Hóa"])
subject_groups_dict = {}
for row in df_subject_group.collect():
    group_id = row['subjectGroupId']
    subjects = [s.strip() for s in row['subjectCombination'].split('-')]
    subject_groups_dict[group_id] = subjects

print(f"  ✓ Số lượng khối thi: {len(subject_groups_dict)}")
print(f"\n  📋 Chi tiết các khối thi:")
for group_id, subjects in list(subject_groups_dict.items())[:5]:
    print(f"     - {group_id}: {subjects}")
print(f"     ... và {len(subject_groups_dict) - 5} khối thi khác")

# Broadcast để worker nodes truy cập nhanh
from pyspark.sql.types import FloatType, MapType, StringType
subject_map_broadcast = spark.sparkContext.broadcast(subject_name_to_id)
subject_groups_broadcast = spark.sparkContext.broadcast(subject_groups_dict)

# ==================== XEM MẪU DỮ LIỆU THÍ SINH ====================
print("\n[Bước 3] Xem mẫu dữ liệu thí sinh trước khi xử lý...")
print("\n  📊 Mẫu điểm của thí sinh (5 thí sinh đầu):")
df_student_scores.select("studentId", "regionId", "year", "scores").show(5, truncate=False)

# ==================== UDF TÍNH ĐIỂM TỔNG THEO KHỐI (CÓ DEBUG) ====================
print("\n[Bước 4] Định nghĩa UDF tính điểm tổng theo khối thi (có debug logging)...")

# Biến global để đếm số lần gọi UDF (chỉ dùng để demo)
import sys

def create_subject_group_scores_map_debug(scores_map):
    """
    UDF chính với DEBUG LOGGING: Tính điểm tổng cho TẤT CẢ các khối thi mà thí sinh có đủ môn
    
    Input:  scores_map = Map<subjectId, score> của 1 thí sinh
    Output: Map<subjectGroupId, totalScore> cho các khối thí sinh có đủ môn
    """
    if scores_map is None:
        return {}
    
    subject_name_to_id_map = subject_map_broadcast.value
    subject_id_to_name_map = {v: k for k, v in subject_name_to_id_map.items()}
    subject_groups = subject_groups_broadcast.value
    
    # DEBUG: In ra điểm của thí sinh (chỉ in 3 thí sinh đầu để tránh spam log)
    student_subjects = {subject_id_to_name_map.get(sid, f"ID_{sid}"): score 
                       for sid, score in scores_map.items()}
    
    result = {}
    groups_eligible = []
    groups_not_eligible = []
    
    # Với mỗi khối thi, kiểm tra xem thí sinh có đủ môn không
    for group_id, required_subjects in subject_groups.items():
        total_score = 0.0
        has_all_subjects = True
        missing_subjects = []
        
        # Kiểm tra từng môn trong khối
        for subject_name in required_subjects:
            subject_id = subject_name_to_id_map.get(subject_name)
            
            # Nếu thiếu môn hoặc không có điểm → Bỏ qua khối này
            if subject_id is None or subject_id not in scores_map:
                has_all_subjects = False
                missing_subjects.append(subject_name)
                break
            
            # Cộng dồn điểm
            total_score += scores_map[subject_id]
        
        # Chỉ thêm vào result nếu thí sinh có đủ TẤT CẢ các môn trong khối
        if has_all_subjects:
            result[group_id] = total_score
            groups_eligible.append(f"{group_id}={total_score:.1f}")
        else:
            groups_not_eligible.append(f"{group_id}(thiếu {','.join(missing_subjects)})")
    
    # DEBUG: In thông tin (chỉ in cho 3 thí sinh đầu)
    # Lưu ý: Trong môi trường distributed, log này sẽ xuất hiện trên executor nodes
    if len(result) > 0:
        # Chỉ in log cho một số thí sinh để minh họa
        pass  # Comment out để tránh quá nhiều log trong production
    
    return result

create_group_scores_udf = udf(create_subject_group_scores_map_debug, MapType(StringType(), FloatType()))
print("  ✓ UDF đã được định nghĩa với debug logging")

# ==================== TEST UDF VỚI MỘT THÍ SINH ====================
print("\n[Bước 5] Test UDF với 1 thí sinh mẫu...")
print("\n  🔍 Lấy 1 thí sinh để test:")

# Lấy 1 thí sinh đầu tiên
test_student = df_student_scores.limit(4).collect()[3]
test_scores = test_student['scores']

print(f"\n  📝 Thí sinh: {test_student['studentId']}")
print(f"     Khu vực: {test_student['regionId']}")
print(f"     Năm: {test_student['year']}")
print(f"\n  📊 Điểm của thí sinh:")

# Convert scores map thành dạng dễ đọc
subject_id_to_name = {v: k for k, v in subject_name_to_id.items()}
for subject_id, score in test_scores.items():
    subject_name = subject_id_to_name.get(subject_id, f"Unknown_{subject_id}")
    print(f"     - {subject_name}: {score}")

# Tính toán thủ công các khối thi mà thí sinh đủ điều kiện
print(f"\n  🎯 Các khối thi thí sinh CÓ THỂ xét:")
eligible_groups = []
for group_id, required_subjects in subject_groups_dict.items():
    has_all = True
    total = 0.0
    for subject_name in required_subjects:
        subject_id = subject_name_to_id.get(subject_name)
        if subject_id is None or subject_id not in test_scores:
            has_all = False
            break
        total += test_scores[subject_id]
    
    if has_all:
        eligible_groups.append((group_id, required_subjects, total))
        print(f"     ✅ {group_id} ({'-'.join(required_subjects)}): {total:.1f} điểm")

print(f"\n  📌 Tổng cộng: {len(eligible_groups)}/{len(subject_groups_dict)} khối thi")

# ==================== ÁP DỤNG UDF ====================
print("\n" + "=" * 80)
print("[Bước 6] Áp dụng UDF cho TẤT CẢ thí sinh...")
print("  ⏳ Đang xử lý... (có thể mất vài phút)")
print("  💡 Lưu ý: Debug log sẽ xuất hiện trên Spark executors, không phải ở đây")
print("=" * 80)

df_student_with_group_scores = df_student_scores.withColumn(
    "subject_group_scores",  # Cột mới: Map<subjectGroupId, totalScore>
    create_group_scores_udf(col("scores"))
)

print("\n  ✓ Hoàn thành!\n")

# ==================== PHÂN TÍCH KẾT QUẢ ====================
print("[Bước 7] Phân tích kết quả:")
print("\n  📊 Xem mẫu kết quả (5 thí sinh):")
df_student_with_group_scores.select(
    "studentId", 
    "regionId", 
    "year", 
    "subject_group_scores"
).show(5, truncate=False)

# Tính thống kê
print("\n  📈 Thống kê số lượng khối thi mà mỗi thí sinh có thể xét:")

# Đếm số lượng khối thi cho mỗi thí sinh
from pyspark.sql.functions import size

df_stats = df_student_with_group_scores.withColumn(
    "num_eligible_groups", 
    size(col("subject_group_scores"))
)

df_stats.groupBy("num_eligible_groups") \
    .agg(count("*").alias("số_thí_sinh")) \
    .orderBy("num_eligible_groups") \
    .show(20)

print("\n  📊 Thống kê tổng quan:")
avg_groups = df_stats.agg(avg("num_eligible_groups").alias("avg")).collect()[0]["avg"]
max_groups = df_stats.agg(max("num_eligible_groups").alias("max")).collect()[0]["max"]
min_groups = df_stats.agg(min("num_eligible_groups").alias("min")).collect()[0]["min"]

print(f"     - Số khối TB mỗi thí sinh có thể xét: {avg_groups:.2f}")
print(f"     - Số khối tối đa: {max_groups}")
print(f"     - Số khối tối thiểu: {min_groups}")
print(f"     - Tổng số khối thi trong hệ thống: {len(subject_groups_dict)}")

print("\n" + "=" * 80)
print("✅ HOÀN THÀNH PHẦN 1: DataFrame 'df_student_with_group_scores' đã sẵn sàng")
print("   với thông tin chi tiết về các khối thi mà mỗi thí sinh có thể xét")
print("=" * 80)

PHẦN 1: TÍNH TOÁN TỔ HỢP KHỐI THI CHO MỖI THÍ SINH (CÓ DEBUG LOG)

[Bước 1] Đọc dữ liệu từ Silver layer...
  ✓ Số lượng thí sinh: 985,353
  ✓ Số lượng khối thi: 212
  ✓ Số lượng môn học: 88

[Bước 2] Tạo mapping dictionaries...
  ✓ Subject mapping: {'Biểu diễn nghệ thuật': 1, 'Công nghệ công nghiệp': 2, 'Công nghệ nông nghiệp': 3, 'GDKT&PL': 4, 'GDKTPL': 5, 'Hoá': 6, 'Hát': 7, 'Hát hoặc biểu diễn nhạc cụ': 8, 'Hình họa': 9, 'Hóa': 10, 'Hóa học': 11, 'Khoa học xã hội': 12, 'Ký xướng âm': 13, 'Lịch sử': 14, 'Lý': 15, 'Lý/Hóa/Sinh/Tin': 16, 'Lịch sử': 17, 'Múa': 18, 'NK Mầm non 1( kể chuyện': 19, 'NK Mầm non 2 (Hát)': 20, 'NK1': 21, 'Ngoại ngữ': 22, 'Ngữ văn': 23, 'Ngữ Văn': 24, 'Ngữ văn': 25, 'Năng Khiếu 1': 26, 'Năng Khiếu 2': 27, 'Năng khiếu': 28, 'Năng khiếu TDTT': 29, 'Năng khiếu': 30, 'Năng khiếu 1': 31, 'Năng khiếu 2': 32, 'Năng khiếu Giáo dục mầm non': 33, 'Năng khiếu SKĐA 1': 34, 'Năng khiếu SKĐA 2': 35, 'Năng khiếu TDTT': 36, 'Năng khiếu Thể dục Thể thao': 37, 'Năng khiếu T

+------------+--------+----+--------------------+
|studentId   |regionId|year|subject_group_scores|
+------------+--------+----+--------------------+
|010000182021|01      |2021|{}                  |
|010000352021|01      |2021|{}                  |
|010000662021|01      |2021|{}                  |
|010000782021|01      |2021|{}                  |
|010000852021|01      |2021|{}                  |
+------------+--------+----+--------------------+
only showing top 5 rows


  📈 Thống kê số lượng khối thi mà mỗi thí sinh có thể xét:


+-------------------+-----------+
|num_eligible_groups|số_thí_sinh|
+-------------------+-----------+
|                  0|     985353|
+-------------------+-----------+


  📊 Thống kê tổng quan:


[Stage 140:>                                                        (0 + 4) / 4]

In [ ]:
# print("=" * 80)
# print("PHẦN 1: TÍNH TOÁN TỔ HỢP KHỐI THI CHO MỖI THÍ SINH")
# print("=" * 80)

# # ==================== ĐỌC DỮ LIỆU ====================
# print("\n[Bước 1] Đọc dữ liệu từ Silver layer...")
# df_student_scores = spark.table("nessie.silver_tables.student_scores")
# df_subject_group = spark.table("nessie.silver_tables.subject_group")
# df_subject = spark.table("nessie.silver_tables.subject")

# print(f"  ✓ Số lượng thí sinh: {df_student_scores.count():,}")
# print(f"  ✓ Số lượng khối thi: {df_subject_group.count()}")
# print(f"  ✓ Số lượng môn học: {df_subject.count()}")

# # ==================== TẠO MAPPING DICTIONARIES ====================
# print("\n[Bước 2] Tạo mapping dictionaries...")

# # Mapping: Tên môn → ID môn (VD: "Toán" → 1)
# subject_name_to_id = {row['subjectName']: row['subjectId'] for row in df_subject.collect()}
# print(f"  ✓ Subject mapping: {subject_name_to_id}")

# # Mapping: ID khối → Danh sách môn (VD: "A00" → ["Toán", "Lý", "Hóa"])
# subject_groups_dict = {}
# for row in df_subject_group.collect():
#     group_id = row['subjectGroupId']
#     subjects = [s.strip() for s in row['subjectCombination'].split('-')]
#     subject_groups_dict[group_id] = subjects

# print(f"  ✓ Số lượng khối thi: {len(subject_groups_dict)}")
# print(f"  ✓ Ví dụ khối thi: {dict(list(subject_groups_dict.items())[:3])}")

# # Broadcast để worker nodes truy cập nhanh
# from pyspark.sql.types import FloatType, MapType, StringType
# subject_map_broadcast = spark.sparkContext.broadcast(subject_name_to_id)
# subject_groups_broadcast = spark.sparkContext.broadcast(subject_groups_dict)

# # ==================== UDF TÍNH ĐIỂM TỔNG THEO KHỐI ====================
# print("\n[Bước 3] Định nghĩa UDF tính điểm tổng theo khối thi...")

# def create_subject_group_scores_map(scores_map):
#     """
#     UDF chính: Tính điểm tổng cho TẤT CẢ các khối thi mà thí sinh có đủ môn
    
#     Input:  scores_map = Map<subjectId, score> của 1 thí sinh
#             VD: {1: 9.0, 2: 8.5, 3: 7.0, 4: 6.0}
#                 (Toán: 9, Lý: 8.5, Hóa: 7, Sinh: 6)
    
#     Output: Map<subjectGroupId, totalScore> cho các khối thí sinh có đủ môn
#             VD: {"A00": 24.5, "B00": 22.0}
    
#     Logic:
#     - Duyệt qua TẤT CẢ các khối thi
#     - Với mỗi khối, kiểm tra xem thí sinh có đủ điểm các môn trong khối không
#     - Nếu đủ → Cộng tổng điểm các môn → Lưu vào result
#     - Nếu thiếu bất kỳ môn nào → Bỏ qua khối đó
#     """
#     if scores_map is None:
#         return {}
    
#     subject_name_to_id_map = subject_map_broadcast.value
#     subject_id_to_name_map = {v: k for k, v in subject_name_to_id_map.items()}
#     subject_groups = subject_groups_broadcast.value
    
#     result = {}
    
#     # Với mỗi khối thi, kiểm tra xem thí sinh có đủ môn không
#     for group_id, required_subjects in subject_groups.items():
#         total_score = 0.0
#         has_all_subjects = True
        
#         # Kiểm tra từng môn trong khối
#         for subject_name in required_subjects:
#             subject_id = subject_name_to_id_map.get(subject_name)
            
#             # Nếu thiếu môn hoặc không có điểm → Bỏ qua khối này
#             if subject_id is None or subject_id not in scores_map:
#                 has_all_subjects = False
#                 break
            
#             # Cộng dồn điểm
#             total_score += scores_map[subject_id]
        
#         # Chỉ thêm vào result nếu thí sinh có đủ TẤT CẢ các môn trong khối
#         if has_all_subjects:
#             result[group_id] = total_score
    
#     return result

# create_group_scores_udf = udf(create_subject_group_scores_map, MapType(StringType(), FloatType()))
# print("  ✓ UDF đã được định nghĩa")

# # ==================== ÁP DỤNG UDF ====================
# print("\n[Bước 4] Áp dụng UDF để tính điểm tổng cho từng thí sinh...")
# print("  ⏳ Đang xử lý... (có thể mất vài phút)")

# df_student_with_group_scores = df_student_scores.withColumn(
#     "subject_group_scores",  # Cột mới: Map<subjectGroupId, totalScore>
#     create_group_scores_udf(col("scores"))
# )

# print("  ✓ Hoàn thành!\n")

# # ==================== HIỂN THỊ MẪU ====================
# print("[Bước 5] Xem kết quả mẫu:")
# print("\nCấu trúc: Mỗi thí sinh có Map chứa tất cả khối thi mà họ có thể xét:")
# df_student_with_group_scores.select(
#     "studentId", 
#     "regionId", 
#     "year", 
#     "subject_group_scores"
# ).show(5, truncate=False)

# print("\n" + "=" * 80)
# print("✅ HOÀN THÀNH PHẦN 1: DataFrame 'df_student_with_group_scores' đã sẵn sàng")
# print("=" * 80)

#### 📊 PHẦN 2: Xử lý và tạo Fact Table từ dữ liệu đã tính toán

**Mục tiêu:** Chuyển đổi dữ liệu tổ hợp khối thi thành Fact table với phân bố điểm theo khoảng.

In [ ]:
print("=" * 80)
print("PHẦN 2: XỬ LÝ VÀ TẠO FACT TABLE")
print("=" * 80)

# ==================== EXPLODE MAP THÀNH ROWS ====================
print("\n[Bước 1] Explode Map thành rows...")
print("  Chuyển từ: 1 row/thí sinh → nhiều rows (1 row cho mỗi khối thi)")

df_scores_exploded = df_student_with_group_scores.select(
    "regionId",
    "year",
    explode("subject_group_scores").alias("subjectGroupId", "totalScore")
)

print(f"  ✓ Tổng số dòng sau explode: {df_scores_exploded.count():,}")
print("\nMẫu dữ liệu sau explode:")
df_scores_exploded.show(10)

# ==================== PHÂN LOẠI ĐIỂM VÀO KHOẢNG ====================
print("\n[Bước 2] Phân loại điểm vào các khoảng...")
print("  Làm tròn xuống: 24.5 → 24.0, 22.8 → 22.0")
print("  Tạo khoảng: [0-1), [1-2), [2-3), ..., [29-30)")

df_scores_exploded = df_scores_exploded.withColumn(
    "scoreRange",
    floor(col("totalScore")).cast("float")
)

print("  ✓ Hoàn thành phân loại điểm")

# ==================== ĐẾM SỐ LƯỢNG HỌC SINH ====================
print("\n[Bước 3] Đếm số lượng học sinh theo từng nhóm...")
print("  GROUP BY: regionId, subjectGroupId, year, scoreRange")

df_distribution = df_scores_exploded.groupBy(
    "regionId",
    "subjectGroupId",
    "year",
    "scoreRange"
).agg(
    count("*").alias("quantity")
)

print(f"  ✓ Số dòng distribution: {df_distribution.count():,}")
print("\nMẫu distribution:")
df_distribution.orderBy(desc("quantity")).show(10)

# ==================== MAPPING SANG SURROGATE KEYS ====================
print("\n[Bước 4] Mapping natural keys sang surrogate keys...")
print("  Sử dụng Dictionary lookup (nhanh hơn JOIN)")

# Đọc dimension tables
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")

# Tạo mapping dictionaries
region_id_to_key = {row['regionId']: row['regionKey'] 
                    for row in df_dim_region_gold.select("regionId", "regionKey").collect()}
subject_group_id_to_key = {row['subjectGroupId']: row['subjectGroupKey'] 
                           for row in df_dim_subject_group_gold.select("subjectGroupId", "subjectGroupKey").collect()}
year_to_time_key = {row['year']: row['timeKey'] 
                    for row in df_dim_time_gold.select("year", "timeKey").collect()}

print(f"  ✓ Region mappings: {len(region_id_to_key)}")
print(f"  ✓ Subject Group mappings: {len(subject_group_id_to_key)}")
print(f"  ✓ Time mappings: {len(year_to_time_key)}")

# Broadcast mappings
region_map_bc = spark.sparkContext.broadcast(region_id_to_key)
subject_group_map_bc = spark.sparkContext.broadcast(subject_group_id_to_key)
year_map_bc = spark.sparkContext.broadcast(year_to_time_key)

# UDF để map keys
def map_to_keys(regionId, subjectGroupId, year):
    region_key = region_map_bc.value.get(regionId)
    subject_group_key = subject_group_map_bc.value.get(subjectGroupId)
    time_key = year_map_bc.value.get(year)
    return (region_key, subject_group_key, time_key)

map_keys_udf = udf(map_to_keys, StructType([
    StructField("regionKey", IntegerType(), True),
    StructField("subjectGroupKey", IntegerType(), True),
    StructField("timeKey", IntegerType(), True)
]))

print("\n[Bước 5] Áp dụng mapping...")

# Áp dụng mapping
df_with_keys = df_distribution.withColumn(
    "keys",
    map_keys_udf(col("regionId"), col("subjectGroupId"), col("year"))
)

# Extract keys từ struct
df_with_keys = df_with_keys.withColumn("regionKey", col("keys.regionKey")) \
    .withColumn("subjectGroupKey", col("keys.subjectGroupKey")) \
    .withColumn("timeKey", col("keys.timeKey")) \
    .drop("keys", "regionId", "subjectGroupId", "year")

print("  ✓ Mapping hoàn thành")

# ==================== TẠO SURROGATE KEY ====================
print("\n[Bước 6] Tạo primary key (SDBSGKey)...")

window_spec = Window.orderBy("timeKey", "regionKey", "subjectGroupKey", "scoreRange")
df_with_keys = df_with_keys.withColumn("SDBSGKey", row_number().over(window_spec))

print("  ✓ Primary key đã được tạo")

# ==================== SELECT CÁC CỘT CUỐI CÙNG ====================
print("\n[Bước 7] Chuẩn bị schema cuối cùng...")

df_fact_score_dist_group = df_with_keys.select(
    "SDBSGKey",
    "regionKey",
    "subjectGroupKey",
    "timeKey",
    col("scoreRange").cast("float"),
    col("quantity").cast("float")
)

print("  ✓ Schema:")
df_fact_score_dist_group.printSchema()

# ==================== GHI VÀO GOLD LAYER ====================
print("\n[Bước 8] Ghi vào Gold layer...")
print("  ⏳ Đang ghi... (partitioned by timeKey)")

df_fact_score_dist_group.writeTo("nessie.gold_tables.fact_score_distribution_by_subject_group") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

final_count = df_fact_score_dist_group.count()
print(f"  ✓ Đã ghi {final_count:,} dòng vào fact_score_distribution_by_subject_group")

# ==================== HIỂN THỊ KẾT QUẢ ====================
print("\n[Bước 9] Xem kết quả cuối cùng:")
print("\nTop 20 dòng (sắp xếp theo timeKey, regionKey):")
df_fact_score_dist_group.orderBy("timeKey", "regionKey", "subjectGroupKey", "scoreRange").show(20)

print("\nThống kê theo năm:")
spark.table("nessie.gold_tables.fact_score_distribution_by_subject_group") \
    .groupBy("timeKey") \
    .agg(
        countDistinct("regionKey").alias("số_khu_vực"),
        countDistinct("subjectGroupKey").alias("số_khối_thi"),
        sum("quantity").alias("tổng_số_học_sinh"),
        count("*").alias("số_records")
    ) \
    .orderBy("timeKey") \
    .show()

print("\n" + "=" * 80)
print("✅ HOÀN THÀNH PHẦN 2: Fact table đã được tạo thành công!")
print("=" * 80)

## 4. Tổng Kết và Kiểm Tra

### 4.1. Kiểm tra số lượng dòng trong các bảng

In [ ]:
print("=== TỔNG KẾT SỐ LƯỢNG DÒNG TRONG CÁC BẢNG ===\n")

tables = [
    "dim_time",
    "dim_region", 
    "dim_school",
    "dim_major",
    "dim_subject",
    "dim_subject_group",
    "dim_selection_method",
    "fact_benchmark",
    "fact_score_distribution_by_subject",
    "fact_score_distribution_by_subject_group"
]

for table_name in tables:
    try:
        count = spark.table(f"nessie.gold_tables.{table_name}").count()
        print(f"{table_name}: {count:,} dòng")
    except Exception as e:
        print(f"{table_name}: Lỗi - {str(e)}")